In [20]:
import camelot
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter    
from openpyxl import Workbook

In [21]:
pdf_file_name = 'test.pdf'
excel_file_name = 'extracted_tables.xlsx'
bank_name = 'Bank0'

In [31]:
tables = camelot.read_pdf(pdf_file_name, pages='all', flavor='stream')

Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)
Ignoring wrong pointing object 1 0 (offset 0)


In [32]:
print(tables)

<TableList n=6>


In [33]:
# Create a new csv file and write tables to separate csv files
for index, table in enumerate(tables):
    table.df.to_csv('table_' + str(index) + '.csv', index=False)

In [34]:
# Create a new Excel workbook and write tables to separate sheets
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
    for index, table in enumerate(tables):
        # Convert table to DataFrame
        df = table.df
        
        # Write each DataFrame to a different sheet
        sheet_name = f'Table_{index + 1}'
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Access the worksheet through writer.sheets
        worksheet = writer.sheets[sheet_name]
        
        # Adjust column widths
        for col_idx, col_cells in enumerate(df.columns, start=1):
            max_length = max(df[col_cells].astype(str).apply(len).max(), col_cells)
            adjusted_width = max_length + 2  # Adding some padding
            worksheet.column_dimensions[get_column_letter(col_idx)].width = adjusted_width

print(f"All tables have been extracted and saved to {excel_file_name}.")

All tables have been extracted and saved to extracted_tables.xlsx.


## Bank 0

In [38]:
user_name = 'SAI DHEERAJ PEKETI'

In [67]:
import re

def is_valid_date(text):
    """
    Checks if the given text matches the DD-MM-YYYY date format.

    Args:
        text (str): The text to validate.

    Returns:
        bool: True if the text matches the date format, False otherwise.
    """
    date_pattern = r'^\d{2}-\d{2}-\d{4}$'
    return bool(re.match(date_pattern, text)) or text == ''

In [72]:
for index, table in enumerate(tables):
    if table.df[0][0] == user_name:
        print(f'Table_{index + 1}: {table.df.shape[1]}')
        current_table_df = table.df.iloc[2:].reset_index(drop=True)
        print(current_table_df.columns)
        table_columns = ['DATE', 'NARRATION', 'CHQ.NO.', 'WITHDRAWAL (DR)', 'DEPOSIT (CR)', 'BALANCE']
        if table.df.shape[1] == 7:
            current_table_df.loc[current_table_df[1] != '', 2] = current_table_df[1]
            current_table_df.drop([1], axis=1, inplace=True)
        elif table.df.shape[1] == 6:
            invalid_dates = ~current_table_df[0].apply(is_valid_date)
            current_table_df.loc[invalid_dates, 1] = current_table_df.loc[invalid_dates, 0]
            current_table_df.loc[invalid_dates, 0] = ''
        current_table_df.reset_index(drop=True, inplace=True)
        current_table_df.columns = table_columns
        current_table_df.to_csv('table_' + str(index) + '.csv', index=False)
            

Table_2: 7
RangeIndex(start=0, stop=7, step=1)
Table_3: 6
RangeIndex(start=0, stop=6, step=1)
Table_4: 7
RangeIndex(start=0, stop=7, step=1)
